In [7]:
import os
import cv2
import re
import numpy as np

In [1]:
# !rm -rf /content/UCF101/UCF101/*

In [2]:
!wget \
  --user=YOUR_USERNAME \
  --password=THUMOS14_REGISTERED \
  --no-check-certificate \
  "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101.zip" \
  -O UCF101.zip

--2025-04-24 02:52:16--  https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101.zip
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 6960400115 (6.5G) [application/zip]
Saving to: ‘UCF101.zip’

UCF101.zip          100%[===================>]   6.48G   147MB/s    in 52s     

2025-04-24 02:53:08 (128 MB/s) - ‘UCF101.zip’ saved [6960400115/6960400115]



In [3]:
!unzip -q UCF101.zip -d UCF101

In [8]:
# 1) Point to your flat folder
UCF_ROOT = "UCF101/UCF101/"  # e.g. /content/UCF101/UCF101/

# 2) Build one sample per class by parsing filenames
def list_ucf_sample_per_class(root=UCF_ROOT):
    samples = {}
    for fname in sorted(os.listdir(root)):
        if not fname.endswith(".avi"):
            continue
        # extract class name between first and second underscore
        # e.g. v_ApplyEyeMakeup_g01_c01.avi → ApplyEyeMakeup
        cls = fname.split("_")[1]
        if cls not in samples:
            samples[cls] = os.path.join(root, fname)
    return samples  # {class: full_path_to_sample.avi}

# 3) Optical-flow motion metric
def compute_avg_flow(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, prev = cap.read()
    if not ret:
        cap.release()
        return 0.0
    prev_gray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
    flows = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(
            prev_gray, curr_gray, None,
            pyr_scale=0.5, levels=3, winsize=15,
            iterations=3, poly_n=5, poly_sigma=1.2, flags=0
        )
        mag, _ = cv2.cartToPolar(flow[...,0], flow[...,1])
        flows.append(np.mean(mag))
        prev_gray = curr_gray
    cap.release()
    return float(np.mean(flows)) if flows else 0.0

# 4) Compute motion for each sampled class
samples = list_ucf_sample_per_class()
motion_stats = []
for cls, path in samples.items():
    avg_flow = compute_avg_flow(path)
    vid = os.path.basename(path)
    motion_stats.append((cls, vid, avg_flow))

# 5) Sort and pick extremes
motion_stats.sort(key=lambda x: x[2])
slowest_cls, slowest_vid, slowest_val = motion_stats[0]
fastest_cls, fastest_vid, fastest_val = motion_stats[-1]

# 6) Report
print(f"Slowest motion class: {slowest_cls}")
print(f"  Video: {slowest_vid}")
print(f"  AvgFlow: {slowest_val:.4f}\n")

print(f"Fastest motion class: {fastest_cls}")
print(f"  Video: {fastest_vid}")
print(f"  AvgFlow: {fastest_val:.4f}")

Slowest motion class: TaiChi
  Video: v_TaiChi_g01_c01.avi
  AvgFlow: 0.0468

Fastest motion class: PoleVault
  Video: v_PoleVault_g01_c01.avi
  AvgFlow: 4.4034


In [11]:
# Download both TaiChi and PoleVault samples into /content/UCF101/UCF101/
!wget \
  --user=YOUR_USERNAME \
  --password=THUMOS14_REGISTERED \
  --no-check-certificate \
  "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_TaiChi_g01_c01.avi" \
  "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_PoleVault_g01_c01.avi"


--2025-04-24 03:12:42--  https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_TaiChi_g01_c01.avi
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 477440 (466K) [video/x-msvideo]
Saving to: ‘v_TaiChi_g01_c01.avi’

v_TaiChi_g01_c01.av 100%[===================>] 466.25K  2.36MB/s    in 0.2s    

2025-04-24 03:12:42 (2.36 MB/s) - ‘v_TaiChi_g01_c01.avi’ saved [477440/477440]

--2025-04-24 03:12:42--  https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/v_PoleVault_g01_c01.avi
Reusing existing connection to www.crcv.ucf.edu:443.
HTTP request sent, awaiting response... 200 OK
Length: 438702 (428K) [video/x-msvideo]
Saving to: ‘v_PoleVault_g01_c01.avi’

v_PoleVault_g01_c01 100%[===================>] 428.42K  --.-KB/s    in 0.002s  

2025-04-24 03:12:42 (271 MB/s) - ‘v_PoleVault_g01_c01.avi’ saved [